# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 12 2023 1:49PM,254944,19,60017908,"GUSA Granules USA, Inc.",Released
1,Jan 12 2023 1:15PM,254938,10,0086295976,ISDIN Corporation,Released
2,Jan 12 2023 1:15PM,254938,10,0086295979,ISDIN Corporation,Released
3,Jan 12 2023 1:15PM,254938,10,0086295975,ISDIN Corporation,Released
4,Jan 12 2023 1:15PM,254938,10,0086295980,ISDIN Corporation,Released
5,Jan 12 2023 1:15PM,254938,10,0086295978,ISDIN Corporation,Released
6,Jan 12 2023 1:15PM,254938,10,0086295977,ISDIN Corporation,Released
7,Jan 12 2023 1:15PM,254938,10,0086295989,ISDIN Corporation,Released
8,Jan 12 2023 1:15PM,254938,10,0086295988,ISDIN Corporation,Released
9,Jan 12 2023 1:14PM,254935,10,MSP217501,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,254922,Released,1
33,254932,Released,2
34,254935,Released,12
35,254938,Released,8
36,254944,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254922,NaN,NaN,1.0
254932,NaN,NaN,2.0
254935,NaN,NaN,12.0
254938,NaN,NaN,8.0
254944,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254814,33.0,0.0,20.0
254819,0.0,0.0,1.0
254821,0.0,0.0,1.0
254824,0.0,0.0,1.0
254825,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254814,33,0,20
254819,0,0,1
254821,0,0,1
254824,0,0,1
254825,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254814,33,0,20
1,254819,0,0,1
2,254821,0,0,1
3,254824,0,0,1
4,254825,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254814,33,,20
1,254819,,,1
2,254821,,,1
3,254824,,,1
4,254825,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 12 2023 1:49PM,254944,19,"GUSA Granules USA, Inc."
1,Jan 12 2023 1:15PM,254938,10,ISDIN Corporation
9,Jan 12 2023 1:14PM,254935,10,"Methapharm, Inc."
21,Jan 12 2023 1:07PM,254932,10,ISDIN Corporation
23,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc."
24,Jan 12 2023 12:31PM,254921,21,"NBTY Global, Inc."
25,Jan 12 2023 12:25PM,254919,10,"Nextsource Biotechnology, LLC"
27,Jan 12 2023 12:22PM,254918,21,"NBTY Global, Inc."
28,Jan 12 2023 12:16PM,254911,10,ISDIN Corporation
42,Jan 12 2023 12:14PM,254915,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 12 2023 1:49PM,254944,19,"GUSA Granules USA, Inc.",,,1
1,Jan 12 2023 1:15PM,254938,10,ISDIN Corporation,,,8
2,Jan 12 2023 1:14PM,254935,10,"Methapharm, Inc.",,,12
3,Jan 12 2023 1:07PM,254932,10,ISDIN Corporation,,,2
4,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc.",,,1
5,Jan 12 2023 12:31PM,254921,21,"NBTY Global, Inc.",,1,
6,Jan 12 2023 12:25PM,254919,10,"Nextsource Biotechnology, LLC",,,2
7,Jan 12 2023 12:22PM,254918,21,"NBTY Global, Inc.",,,1
8,Jan 12 2023 12:16PM,254911,10,ISDIN Corporation,,,14
9,Jan 12 2023 12:14PM,254915,10,"Citieffe, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 1:49PM,254944,19,"GUSA Granules USA, Inc.",1,,
1,Jan 12 2023 1:15PM,254938,10,ISDIN Corporation,8,,
2,Jan 12 2023 1:14PM,254935,10,"Methapharm, Inc.",12,,
3,Jan 12 2023 1:07PM,254932,10,ISDIN Corporation,2,,
4,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc.",1,,
5,Jan 12 2023 12:31PM,254921,21,"NBTY Global, Inc.",,1,
6,Jan 12 2023 12:25PM,254919,10,"Nextsource Biotechnology, LLC",2,,
7,Jan 12 2023 12:22PM,254918,21,"NBTY Global, Inc.",1,,
8,Jan 12 2023 12:16PM,254911,10,ISDIN Corporation,14,,
9,Jan 12 2023 12:14PM,254915,10,"Citieffe, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 1:49PM,254944,19,"GUSA Granules USA, Inc.",1,,
1,Jan 12 2023 1:15PM,254938,10,ISDIN Corporation,8,,
2,Jan 12 2023 1:14PM,254935,10,"Methapharm, Inc.",12,,
3,Jan 12 2023 1:07PM,254932,10,ISDIN Corporation,2,,
4,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 1:49PM,254944,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Jan 12 2023 1:15PM,254938,10,ISDIN Corporation,8.0,NaN,NaN
2,Jan 12 2023 1:14PM,254935,10,"Methapharm, Inc.",12.0,NaN,NaN
3,Jan 12 2023 1:07PM,254932,10,ISDIN Corporation,2.0,NaN,NaN
4,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 1:49PM,254944,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Jan 12 2023 1:15PM,254938,10,ISDIN Corporation,8.0,0.0,0.0
2,Jan 12 2023 1:14PM,254935,10,"Methapharm, Inc.",12.0,0.0,0.0
3,Jan 12 2023 1:07PM,254932,10,ISDIN Corporation,2.0,0.0,0.0
4,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2549164,73.0,3.0,0.0
15,509680,21.0,0.0,33.0
16,254874,1.0,0.0,0.0
17,254899,1.0,0.0,0.0
19,1274488,20.0,29.0,18.0
21,2803407,10.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2549164,73.0,3.0,0.0
1,15,509680,21.0,0.0,33.0
2,16,254874,1.0,0.0,0.0
3,17,254899,1.0,0.0,0.0
4,19,1274488,20.0,29.0,18.0
5,21,2803407,10.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,73.0,3.0,0.0
1,15,21.0,0.0,33.0
2,16,1.0,0.0,0.0
3,17,1.0,0.0,0.0
4,19,20.0,29.0,18.0
5,21,10.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,73.0
1,15,Released,21.0
2,16,Released,1.0
3,17,Released,1.0
4,19,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,19,21
Status,,,,,,
Completed,0.0,33.0,0.0,0.0,18.0,0.0
Executing,3.0,0.0,0.0,0.0,29.0,1.0
Released,73.0,21.0,1.0,1.0,20.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,19,21
0,Completed,0.0,33.0,0.0,0.0,18.0,0.0
1,Executing,3.0,0.0,0.0,0.0,29.0,1.0
2,Released,73.0,21.0,1.0,1.0,20.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,19,21
0,Completed,0.0,33.0,0.0,0.0,18.0,0.0
1,Executing,3.0,0.0,0.0,0.0,29.0,1.0
2,Released,73.0,21.0,1.0,1.0,20.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()